## Configuration and Setup


In [1]:
import os
os.environ['HF_HOME'] = "C:/HF_CACHE/"

In [2]:
import torch

from cs324_project.datasets import GlueDatasetTask, load_glue_dataset_info
from cs324_project.models import ModelCheckpointName, load_classification_model, load_pretraining_model, load_tokenizer
from cs324_project.masking import (
    get_training_args_mlm, get_trainer_mlm, RandomMaskingConfig, WholeWordMaskingConfig, TyphoonMaskingConfig)
from cs324_project.classification import get_training_args_sc, get_trainer_sc

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
model_name = ModelCheckpointName.TINYBERT_HUAWEI
task = GlueDatasetTask.MRPC

In [5]:
tokenizer = load_tokenizer(model_name)
dataset_info = load_glue_dataset_info(task, tokenizer)

Found cached dataset glue (C:/HF_CACHE/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-210d87aa331c9a9e.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-33abd690c2474663.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-fd99a719fa52af0a.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-98cf444ee3350795.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-9383850b1455b190.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-bcdcf9ec1f3ace0e.arro

## Fine-tune with masking

In [6]:
model_mlm = load_pretraining_model(model_name, dataset_info)

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertForMaskedLM: ['fit_denses.1.bias', 'cls.seq_relationship.weight', 'fit_denses.3.weight', 'fit_denses.4.bias', 'fit_denses.2.bias', 'fit_denses.2.weight', 'fit_denses.0.weight', 'fit_denses.0.bias', 'cls.seq_relationship.bias', 'fit_denses.3.bias', 'fit_denses.1.weight', 'fit_denses.4.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
training_args_mlm = get_training_args_mlm(
    masking_config=WholeWordMaskingConfig(),
    num_epochs=100)
trainer_mlm = get_trainer_mlm(
    dataset_info=dataset_info,
    mlm_args=training_args_mlm,
    model=model_mlm)

Creating training arguments, model output dir: C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\mlm\Model 03-22-2023 05-57-09 AM


In [8]:
trainer_mlm.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,6.390300,5.571664
2,5.267300,4.936996
3,4.775700,4.635204
4,4.449600,4.365764
5,4.236900,4.239106
6,4.051600,4.056751
7,3.928400,3.977224
8,3.772500,3.912984
9,3.670600,3.826396
10,3.579800,3.769111


TrainOutput(global_step=23000, training_loss=2.404684521219005, metrics={'train_runtime': 1670.9205, 'train_samples_per_second': 219.52, 'train_steps_per_second': 13.765, 'total_flos': 786121535232432.0, 'train_loss': 2.404684521219005, 'epoch': 100.0})

In [9]:
trainer_mlm.evaluate()

{'eval_loss': 3.3562185764312744,
 'eval_runtime': 0.5924,
 'eval_samples_per_second': 688.711,
 'eval_steps_per_second': 43.888,
 'epoch': 100.0}

In [10]:
print(trainer_mlm.state.best_model_checkpoint)

C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\mlm\Model 03-22-2023 05-57-09 AM\checkpoint-10810


## Fine-tune on sequence classification task

In [11]:
model_sc = load_classification_model(trainer_mlm.state.best_model_checkpoint, dataset_info)

Some weights of the model checkpoint at C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\mlm\Model 03-22-2023 05-57-09 AM\checkpoint-10810 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequ

In [12]:
training_args_sc = get_training_args_sc(
    task,
    learning_rate=1e-5,
    num_epochs=100)
trainer_sc = get_trainer_sc(
    dataset_info=dataset_info,
    model=model_sc,
    training_args=training_args_sc)

Creating training arguments, model output dir: C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\sc\Model 03-22-2023 06-25-01 AM


In [13]:
trainer_sc.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.604500,0.550685,0.715686,0.826347
2,0.521300,0.493689,0.769608,0.848387
3,0.455700,0.473383,0.774510,0.838596
4,0.401900,0.464810,0.759804,0.815789
5,0.363200,0.483908,0.774510,0.820312
6,0.330200,0.490200,0.776961,0.825336
7,0.294300,0.558731,0.740196,0.788845
8,0.259100,0.607836,0.752451,0.801572
9,0.234800,0.647670,0.745098,0.794466
10,0.215300,0.786610,0.742647,0.787018


TrainOutput(global_step=23000, training_loss=0.06476597296155018, metrics={'train_runtime': 989.4777, 'train_samples_per_second': 370.701, 'train_steps_per_second': 23.245, 'total_flos': 781014765977712.0, 'train_loss': 0.06476597296155018, 'epoch': 100.0})

In [14]:
trainer_sc.evaluate()

{'eval_loss': 0.4901997148990631,
 'eval_accuracy': 0.7769607843137255,
 'eval_f1': 0.8253358925143953,
 'eval_runtime': 0.367,
 'eval_samples_per_second': 1111.665,
 'eval_steps_per_second': 70.841,
 'epoch': 100.0}

In [15]:
print(trainer_sc.state.best_model_checkpoint)

C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\sc\Model 03-22-2023 06-25-01 AM\checkpoint-1380
